In [2]:
import cfgrib
import xarray as xr

import pandas as pd
import numpy as np

from pyPhenology import models, utils
import numpy as np

from tqdm import trange, tqdm

import matplotlib.pyplot as plt

from warnings import warn

In [22]:
def ripeness_data_to_dict(ripeness_data):
    final_ripeness_data = ripeness_data[(ripeness_data.species == species) & (ripeness_data.site_id == site_id)]
    
    mean_maturation = np.mean(final_ripeness_data['flowering_day'])
    
    prediction_dict = {
        "full_flowering_data": final_ripeness_data,
        "species_site_flowering days": list(final_ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(final_ripeness_data['flowering_day'])
    }
    
    return prediction_dict

In [23]:
def aic(obs, pred, n_param):
        return len(obs) * np.log(np.mean((obs - pred)**2)) + 2*(n_param + 1)

observations, predictors = utils.load_test_data(name='vaccinium',
                                                phenophase='budburst')

default_models = [models.ThermalTime(), models.FallCooling(), models.M1(), models.MSB()]

default_model_names = ['ThermalTime', "FallCooling", "M1", "MSB"]

def train_ripeness(observations, predictors, test_observations, test_predictors, models=['ThermalTime']):

    
    # set up model comparisons
    best_aic=np.inf
    best_model = None
    best_model_name = None

    # iterate through all models
    for model_name in models:
        print("running model {m}".format(m=model_name))
        
        Model = utils.load_model(model_name)
        model = Model()
        model.fit(observations, predictors, optimizer_params='practical')
        
        # predict from test observations
        print("making predictions for model {m}".format(m=model_name))        
        preds = model.predict(test_observations, test_predictors)
        
        #print(preds)
        test_days = test_observations.doy.values
        #print(test_days)
        # this isn't valid - need to filter by site IDs
        
        # THIS IS REALLY BAD:
        test_days = test_days[0:len(preds)]
        #print(test_days)
        
        # score model
        model_aic = aic(obs = test_days,
                        pred=preds,
                        n_param = len(model.get_params()))
        print(model_aic)

        if model_aic < best_aic:
            best_model = model
            best_model_name = model_name
            best_aic = model_aic

        print('model {m} got an aic of {a}'.format(m=model_name,a=model_aic))

    print('Best model: {m}'.format(m=best_model_name))
    print('Best model paramters:')
    print(best_model.get_params())
    
    ripeness_data = test_observations
    ripeness_data['flowering_day'] = preds
    
    return ripeness_data

In [25]:
def train_ripeness_percent(observations, predictors, test_percent, models=['ThermalTime']):
    test_observations = observations.sample(frac=test_percent)
    observations_train = observations.drop(test_observations.index)
    
    # set up model comparisons
    best_aic=np.inf
    best_model = None
    best_model_name = None

    # iterate through all models
    for model_name in models:
        print("running model {m}".format(m=model_name))
        
        Model = utils.load_model(model_name)
        model = Model()
        model.fit(observations_train, predictors, optimizer_params='practical')
        
        # predict from test observations
        print("making predictions for model {m}".format(m=model_name))        
        preds = model.predict(test_observations, predictors)
    
        #print(preds)
        test_days = test_observations.doy.values
        #print(test_days)
        
        # THIS IS REALLY BAD:
        test_days = test_days[0:len(preds)]
        #print(test_days)
        
        # score model
        model_aic = aic(obs = test_days,
                        pred=preds,
                        n_param = len(model.get_params()))
        print(model_aic)

        if model_aic < best_aic:
            best_model = model
            best_model_name = model_name
            best_aic = model_aic

        print('model {m} got an aic of {a}'.format(m=model_name,a=model_aic))

    print('Best model: {m}'.format(m=best_model_name))
    print('Best model paramters:')
    print(best_model.get_params())
    
    ripeness_data = test_observations
    ripeness_data['flowering_day'] = preds
    
    return ripeness_data

In [5]:
# Load and process weather data

grib_data = cfgrib.open_datasets('../data/weather_data.grib')

core_data = grib_data[0]

In [40]:
# Load and process apple data
apple_data = pd.read_csv('../data/plant phenology/malus.csv')

apple_data["lon_360"] = apple_data["LON"] % 360

# This min max is derived from a historgram – think of how to do this algorithmically.
concentrated_apples = apple_data[(apple_data['lon_360'] >= 5) & (apple_data['lon_360'] <= 17)]

concentrated_apples = concentrated_apples[concentrated_apples['YEAR'] >= 2010].drop_duplicates()

In [7]:
concentrated_weather = core_data.where((core_data.latitude <= 55) & 
                                       (core_data.latitude >= 44) &
                                        (core_data.longitude <= 18) & 
                                       (core_data.longitude >= 5), drop=True)

In [8]:
def get_site_history(weather_array, site_id, site_lat, site_lon):
    filtered = weather_array.where((abs(weather_array.latitude - site_lat) <= 0.05) & (abs(weather_array.longitude - site_lon) <= 0.05), drop=True)
    
    #print("Converting GRIB to dataframe")
    site_df = filtered.to_dataframe().drop(["number", "step", "surface"], axis=1).reset_index().rename(columns={"skt":"temperature"})
    
    site_df['site_id'] = site_id
    
    site_df['year'] = site_df.time.dt.to_period('Y')
    site_df['doy'] = site_df.time.dt.strftime('%j').astype(int)
    
    site_df = site_df[['site_id', 'temperature', 'year', 'doy', 'latitude', 'longitude']]
    
    return(site_df)

In [9]:
# get site histories

site_histories = []

for index, row in tqdm(concentrated_apples.iterrows()):
    
    site_histories.append(get_site_history(concentrated_weather, row['site_id'], row['LAT'], row['lon_360']))

# create site history df, process a bit
full_site_histories = pd.concat(site_histories).dropna()

full_site_histories['year'] = full_site_histories['year'].astype(str).astype(int)
full_site_histories['site_id'] = full_site_histories['site_id'].astype(int)

# Correct for leap years
leap_year_key = {60: 61, 
                 91: 92, 
                 121: 122, 
                 152: 153, 
                 182: 183, 
                 213: 214, 
                 244: 245, 
                 274: 275, 
                 305: 306, 
                 335: 336}

corrected_leap_year_histories = full_site_histories.replace({'doy': leap_year_key})

10613it [02:08, 82.73it/s]


In [43]:
def claudia_observations_to_pyphenology(claudia_obs):
    new_observations = claudia_obs.copy(deep=True)
    
    new_observations['species_actual'] = new_observations['specificEpithet']
    
    new_observations.rename(columns={'YEAR': 'year',
                            'DAY': 'doy',
                            'genus': 'species',
                            'LAT': 'latitude'}, inplace=True)
    
    new_observations.drop(['specificEpithet', 'eventRemarks', 'LON', 'lon_360'], axis=1, inplace=True)
    
    new_observations['phenophase'] = 516
    
    return new_observations

In [50]:
example_observations_2 = claudia_observations_to_pyphenology(concentrated_apples)

In [54]:
filtered_observations = example_observations_2[example_observations_2['site_id'].isin(corrected_leap_year_histories['site_id'])]
filtered_observations.dropna(inplace=True)

/var/folders/xk/t3y_dz7j75ld1xlbfffyj5lm0000gn/T/ipykernel_37537/899493858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_observations.dropna(inplace=True)


In [26]:
# Drop sites where there's no weather data
raw_preds = train_ripeness_percent(filtered_observations, corrected_leap_year_histories, 0.2)

running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


making predictions for model ThermalTime
14867.730304862449
model ThermalTime got an aic of 14867.730304862449
Best model: ThermalTime
Best model paramters:
{'t1': 161.11544149067402, 'T': 12.383106307297997, 'F': 857.6778689783685}


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


In [34]:
len(raw_preds['site_id'].unique())

1023

In [32]:
np.mean(raw_preds['doy'])

248.34034416826003

In [33]:
np.std(raw_preds['doy'])

34.71269456459817

### Separate data into two groups: above 245 and below 245

In [56]:
filtered_observations_high_days = filtered_observations[filtered_observations['doy'] >= 245]
filtered_observations_low_days = filtered_observations[filtered_observations['doy']<= 245]

In [58]:
print(train_ripeness_percent(filtered_observations_low_days, corrected_leap_year_histories, 0.2))
print(train_ripeness_percent(filtered_observations_high_days, corrected_leap_year_histories, 0.2))


running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


making predictions for model ThermalTime
5152.7876757856475
model ThermalTime got an aic of 5152.7876757856475
Best model: ThermalTime
Best model paramters:
{'t1': 205.35806282612515, 'T': -6.676849196936063, 'F': 143.05545908351883}
        year  doy species  latitude  site_id              species_actual  \
100635  2015  205   Malus   49.9667     6161  domestica 'early cultivar'   
100982  2013  213   Malus   53.0833     6209  domestica 'early cultivar'   
43922   2015  226   Malus   50.2667     2525  domestica 'early cultivar'   
100428  2014  217   Malus   50.4333     6142  domestica 'early cultivar'   
40730   2014  223   Malus   48.4000     2366  domestica 'early cultivar'   
...      ...  ...     ...       ...      ...                         ...   
74858   2013  220   Malus   54.3000     4413  domestica 'early cultivar'   
89274   2011  210   Malus   52.4000     5298  domestica 'early cultivar'   
84711   2011  195   Malus   51.2167     3595  domestica 'early cultivar'   
61720 

/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


making predictions for model ThermalTime
5887.62136978343
model ThermalTime got an aic of 5887.62136978343
Best model: ThermalTime
Best model paramters:
{'t1': 208.85852600214034, 'T': -3.3955265217295083, 'F': 700.2866793209392}
        year  doy species  latitude  site_id              species_actual  \
267575  2013  280   Malus   53.4833     2876   domestica 'late cultivar'   
216201  2014  256   Malus   49.1833     1349   domestica 'late cultivar'   
321938  2012  285   Malus   51.8333     6218   domestica 'late cultivar'   
236284  2012  271   Malus   53.5833     1175   domestica 'late cultivar'   
106397  2010  269   Malus   47.1869     6659   domestica 'late cultivar'   
...      ...  ...     ...       ...      ...                         ...   
245992  2010  289   Malus   49.4000     1429   domestica 'late cultivar'   
301614  2011  287   Malus   48.9667     4820   domestica 'late cultivar'   
317007  2013  283   Malus   48.0833     5750   domestica 'late cultivar'   
101348  20

/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 336 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


In [63]:
filtered_observations_high_days['species_actual'].value_counts()

species_actual
domestica 'late cultivar'         5218
domestica 'early cultivar'         213
domestica                          111
pumila                              21
domestica 'Kronprinz'               16
domestica 'Golden delicius'         12
domestica 'Jonagold'                 9
domestica 'Idared'                   6
domestica 'Roter boskoop'            5
domestica 'Jonathan'                 4
domestica 'Gravensteiner'            4
domestica 'middle cultivar'          3
domestica 'James grieve'             3
domestica 'Cox Orange Renette'       2
domestica 'Goldparmane'              2
domestica 'Rubinette'                2
domestica 'Gloster'                  1
domestica 'Elstar'                   1
pumila 'Gala'                        1
pumila 'White Transparent'           1
Name: count, dtype: int64

In [62]:
filtered_observations_low_days['species_actual'].value_counts()

species_actual
domestica 'early cultivar'        4657
domestica 'late cultivar'          114
domestica                           41
pumila                              11
domestica 'Gravensteiner'           10
domestica 'Kronprinz'                4
domestica 'Cox Orange Renette'       2
pumila 'Gala'                        2
pumila 'Api Rouge sur Franc'         2
domestica 'Weisser klarapfel'        2
domestica 'middle cultivar'          1
domestica 'Goldparmane'              1
domestica 'James grieve'             1
pumila 'Pomme Gris'                  1
pumila 'Cox's Orange Pippin'         1
Name: count, dtype: int64

Early cultivar ready in August, late cultivar ready in October.